# Problem Set 7 — Coding Part

**This problem set is part of the course "Data Compression With Deep Probabilistic Models" by Prof. Robert Bamler at University of Tuebingen, Germany. You can find more course materials (lecture notes, video recordings, and solutions) at the course website, https://robamler.github.io/teaching/compress21/**

**Problem Set Published:** 9 June 2021<br>
**Discussion:** 14 June 2021

Please see accompanying PDF document for instructions.

## Naive ANS Coder from the lecture

Below is a copy of the implementation of the ANS coder that we implemented in the lecture.
While it is a logically correct entropy coder, it is not yet quite useful because, as discussed at the end of the lecture, its runtime scales quadratically in the number of encoded symbols.

In [1]:
class AnsCoder:
    def __init__(self, precision):
        self.precision = precision
        self.mask = (1 << precision) - 1
        self.compressed = 1

    def encode(self, symbol, scaled_probabilities):
        z = self.compressed % scaled_probabilities[symbol]
        self.compressed //= scaled_probabilities[symbol]
        for prob in scaled_probabilities[:symbol]:
            z += prob
        self.compressed = (self.compressed << self.precision) | z

    def decode(self, scaled_probabilities):
        z = self.compressed & self.mask
        self.compressed >>= self.precision

        for i, prob in enumerate(scaled_probabilities):
            if prob > z:
                symbol = i
                break
            else:
                z -= prob

        self.compressed = self.compressed * scaled_probabilities[symbol] + z
        return symbol

### Our tests from the lecture

In [2]:
# n = 2**precision
precision = 4 # ==> n = 16
coder = AnsCoder(precision)

scaled_probabilities1 = [3, 7, 2, 4]
scaled_probabilities2 = [8, 2, 2, 4]
scaled_probabilities3 = [1, 5, 3, 3, 4]

coder.encode(1, scaled_probabilities1)
coder.encode(0, scaled_probabilities2)
coder.encode(4, scaled_probabilities3)

print(f'{coder.compressed:b}')

print(coder.decode(scaled_probabilities3))
print(coder.decode(scaled_probabilities2))
print(coder.decode(scaled_probabilities1))
# Should print encoded symbols in reverse order.

11100
4
0
1


## Problem 7.1

Below is a skeleton implementation of a `StreamingAnsCoder` class.
Follow the instructions in the accompanying PDF document to fill in the missing parts.

In [3]:
class StreamingAnsCoder:
    def __init__(self, precision):
        # YOUR TASK (Problem 7.1 (e)): add an (optional) parameter `compressed` that
        # accepts an initial compressed representation from which we can then decode.
        self.precision = precision
        self.mask = (1 << precision) - 1
        self.bulk = []
        self.head = 1 # We could technically initialize this with zero too.

    def encode(self, symbol, scaled_probabilities):
        # YOUR TASK (Problem 7.1 (c)): uphold invariants for `self.head`
        z = self.head % scaled_probabilities[symbol]
        self.head //= scaled_probabilities[symbol]
        for prob in scaled_probabilities[:symbol]:
            z += prob
        self.head = (self.head << self.precision) | z

    def decode(self, scaled_probabilities):
        # YOUR TASK (Problem 7.1 (d)): uphold invariants for `self.head` and make sure
        # `self.decode` exactly inverts `self.encode`
        z = self.head & self.mask
        self.head >>= self.precision

        for i, prob in enumerate(scaled_probabilities):
            if prob > z:
                symbol = i
                break
            else:
                z -= prob

        self.head = self.head * scaled_probabilities[symbol] + z        
        return symbol
    
    def get_compressed(self):
        # YOUR TASK (Problem 7.1 (e)): return the compressed representation as a list
        # of integers with `self.precision` bits each.
        pass

### Test for Problems 7.1 (c) and (d)

Use this test (and possibly some additional tests of your own) to debug your implementations of the `encode` and `decode` methods.

In [4]:
import numpy as np

In [5]:
def random_model_and_symbol(seed, precision):
    """Creates a reproducible pseudorandom model and draws a reproducible pseudorandom symbol from it."""
    rng = np.random.RandomState(seed)
    alphabet_size = 2 + rng.choice(10)

    # Ensure that all scaled_probabilities are nonzero and that they add up to `(1 << precision)`
    assert alphabet_size <= (1 << precision)
    scaled_probabilities = ((1 << precision) * rng.dirichlet([1] * alphabet_size)).astype(np.int64) + 1
    for _ in range(scaled_probabilities.sum() - (1 << precision)):
        scaled_probabilities[scaled_probabilities.argmax()] -= 1

    # Draw a random symbol and calculate its information content
    symbol = rng.choice(alphabet_size, p=(1/(1 << precision)) * scaled_probabilities)
    inf_content = precision - np.log2(scaled_probabilities[symbol])
    return scaled_probabilities, symbol, inf_content

In [6]:
precision = 12
num_symbols = 1000
master_seed = 123

coder = StreamingAnsCoder(precision)
total_inf_content = 0
for i in range(num_symbols):
    scaled_probabilities, symbol, inf_content = random_model_and_symbol(
        master_seed * num_symbols + i, precision)
    coder.encode(symbol, scaled_probabilities)
    total_inf_content += inf_content

bitrate = (len(coder.bulk) + 2) * precision
print(f'Encoded {num_symbols} random symbols with a total information content of ' +
      f'{total_inf_content:.2f} bits into {bitrate} bits.')
print(f'- absolute overhead: {bitrate - total_inf_content:.2f} bits')
print(f'- relative overhead: {100 * (bitrate - total_inf_content) / total_inf_content:.2g}% (expect about 1%)')

for i in reversed(range(num_symbols)):
    scaled_probabilities, expected_symbol, _ = random_model_and_symbol(
        master_seed * num_symbols + i, precision)
    symbol = coder.decode(scaled_probabilities)
    assert symbol == expected_symbol

print('Successfully reconstructed original message.')

Encoded 1000 random symbols with a total information content of 2001.55 bits into 2016 bits.
- absolute overhead: 14.45 bits
- relative overhead: 0.72% (expect about 1%)
Successfully reconstructed original message.


### Test for Problem 7.1 (e)

Use this test (and possibly some additional tests of your own) to debug your implementations of the `get_compressed` method and the constructor.

In [7]:
precision = 12
num_symbols = 1000
master_seed = 456

encoder = StreamingAnsCoder(precision)
total_inf_content = 0
for i in range(num_symbols):
    scaled_probabilities, symbol, inf_content = random_model_and_symbol(
        master_seed * num_symbols + i, precision)
    encoder.encode(symbol, scaled_probabilities)
    total_inf_content += inf_content

compressed = encoder.get_compressed()
bitrate = (len(encoder.bulk) + 2) * precision
print(f'Encoded {num_symbols} random symbols with a total information content of ' +
      f'{total_inf_content:.2f} bits into {bitrate} bits.')
print(f'- absolute overhead: {bitrate - total_inf_content:.2f} bits')
print(f'- relative overhead: {100 * (bitrate - total_inf_content) / total_inf_content:.2g}% (expect about 1%)')

decoder = StreamingAnsCoder(precision, compressed)
for i in reversed(range(num_symbols)):
    scaled_probabilities, expected_symbol, _ = random_model_and_symbol(
        master_seed * num_symbols + i, precision)
    symbol = decoder.decode(scaled_probabilities)
    assert symbol == expected_symbol

print('Successfully reconstructed original message.')

Encoded 1000 random symbols with a total information content of 2101.30 bits into 2124 bits.
- absolute overhead: 22.70 bits
- relative overhead: 1.1% (expect about 1%)
Successfully reconstructed original message.
